In [1]:
import os
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.utils.data as data
import torchvision
import torch.optim as optim

from model.effb7_unet import EFFUnet
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Loaded pretrained weights for efficientnet-b7


In [2]:
class DFDCDataset(Dataset):
    def __init__(self, img_dir, labels_dir):
        super().__init__()
        self.img_dir = img_dir
        self.labels_dir = labels_dir

        self.names = []
        for i in os.listdir(img_dir):
            if i in os.listdir(labels_dir):
                self.names.append(i)

    def __len__(self):
        return len(self.names)

    def load(self, dir_path, name, mode):
        if mode == "img":
            return torch.from_numpy(np.array(Image.open(os.path.join(dir_path, name))))
        else:
            return torch.from_numpy(np.array(Image.open(os.path.join(dir_path, name)).convert('L')))

    def __getitem__(self, index):
        selected = self.names[index]

        img = self.load(self.img_dir, selected, "img")
        label = self.load(self.labels_dir, selected, "label")

        img = img/255

        # replacing pixel values by the classes
        label[label<170] = 0
        label[label>=170] = 1        

        return img.float().permute(2,0,1).contiguous(), label.float().contiguous()

In [3]:
dataset = DFDCDataset("mini_dataset/crops", "mini_dataset/diffs")
train_set_size = int(len(dataset) * 0.8)
valid_set_size = len(dataset) - train_set_size
train_set, valid_set = random_split(dataset, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(42))

In [4]:
trainloader = torch.utils.data.DataLoader(train_set, batch_size=1,
                                          shuffle=True, num_workers=0)
validloader = torch.utils.data.DataLoader(valid_set, batch_size=1,
                                         shuffle=False, num_workers=0)

In [5]:
# import matplotlib.pyplot as plt
# import numpy as np

# # functions to show an image


# def imshow(img):    # unnormalize
#     npimg = img.numpy()
#     plt.imshow(npimg)
#     plt.show()


# # get some random training images
# dataiter = iter(trainloader)
# images, labels = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# # print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

In [6]:
net = EFFUnet(n_classes=2)
net.to(device=device)

grad_scaler = torch.cuda.amp.GradScaler(enabled=True)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=1e-4)

Loaded pretrained weights for efficientnet-b7


In [7]:
epoch = 50
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(inputs.shape)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels.long())
        # loss.backward()
        # optimizer.step()
        grad_scaler.scale(loss).backward()
        grad_scaler.step(optimizer)
        grad_scaler.update()

        # print statistics
        running_loss += loss.item()
        if i % 1 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss))
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.719
[1,     2] loss: 0.425
[1,     3] loss: 0.335
[1,     4] loss: 0.332
[1,     5] loss: 0.320
[1,     6] loss: 0.322
[1,     7] loss: 0.410
[1,     8] loss: 0.330
[1,     9] loss: 0.336
[1,    10] loss: 0.395
[1,    11] loss: 0.327
[1,    12] loss: 0.321
[1,    13] loss: 0.405
[1,    14] loss: 0.336
[1,    15] loss: 0.313
[1,    16] loss: 0.347
[1,    17] loss: 0.331
[1,    18] loss: 0.320
[1,    19] loss: 0.330
[1,    20] loss: 0.316
[1,    21] loss: 0.322
[1,    22] loss: 0.329
[1,    23] loss: 0.321
[1,    24] loss: 0.344
[1,    25] loss: 0.316
[1,    26] loss: 0.322
[1,    27] loss: 0.324
[1,    28] loss: 0.313
[1,    29] loss: 0.313
[1,    30] loss: 0.324
[1,    31] loss: 0.319
[1,    32] loss: 0.318
[1,    33] loss: 0.321
[1,    34] loss: 0.332
[1,    35] loss: 0.319
[1,    36] loss: 0.313
[1,    37] loss: 0.313
[1,    38] loss: 0.313
[1,    39] loss: 0.313
[1,    40] loss: 0.322
[1,    41] loss: 0.324
[1,    42] loss: 0.313
[1,    43] loss: 0.317
[1,    44] 

In [13]:
torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, "first_model.pth")

In [7]:
# model = EFFUnet(n_classes=2)
# # optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.001)

checkpoint = torch.load("first_model.pth")
net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

net.eval()

EFFUnet(
  (effunet_encoder): EFFUnet_encoder()
  (effunet_decoder): EFFUnet_decoder(
    (upconvolution_1): Upconvolution(
      (upconv): ConvTranspose2d(640, 512, kernel_size=(2, 2), stride=(2, 2))
    )
    (upconvolution_2): Upconvolution(
      (upconv): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
    )
    (upconvolution_3): Upconvolution(
      (upconv): ConvTranspose2d(256, 128, kernel_size=(2, 2), stride=(2, 2))
    )
    (upconvolution_4): Upconvolution(
      (upconv): ConvTranspose2d(128, 64, kernel_size=(2, 2), stride=(2, 2))
    )
    (upconvolution_5): Upconvolution(
      (upconv): ConvTranspose2d(64, 16, kernel_size=(2, 2), stride=(2, 2))
    )
    (conv_1): Conv2d(896, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv_2): Conv2d(416, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv_3): Conv2d(208, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv_4): Conv2d(112, 64, kernel_size=(3, 3), stride=(1, 1), p

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model = model.to(device)
for i, data in enumerate(validloader, 10):
    inputs, labels = data
    inputs = inputs.to(device)
    outputs = net(inputs)
    print(outputs)
    print(labels)
    break

tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [9]:
notmanipulated, manipulated = outputs[0]
print(np.unique(notmanipulated.cpu().detach().numpy()))
print(np.unique(manipulated.cpu().detach().numpy()))

[1.]
[0.]


In [29]:
output = notmanipulated*0 + manipulated
np.unique(output.cpu().detach().numpy())

array([0.], dtype=float32)

In [27]:
output[output == 1.] = 255

In [28]:
np.unique(output.cpu().detach().numpy())

array([0.], dtype=float32)